In [ ]:
import ee, pandas as pd

def _glacier_metrics_for_lake_polygon(lake, buffer_km=50):
    """
    Annotate ONE lake polygon with glacier metrics, fully server-side.

    Rules:
      - Use only glaciers that are strictly higher than the lake (by DEM).
      - Exclude touching/overlapping glaciers (dist == 0).
      - Pick the nearest among those.
      - Compute slope along the true min-distance line using DEM at the two closest points.
      - If no valid glacier exists -> return null-like fields.
    """
    glaciers = ee.FeatureCollection("GLIMS/current")
    dem      = ee.Image("USGS/SRTMGL1_003")

    # Ensure non-zero-area geometry (handles accidental points)
    lake_geom = ee.Feature(lake).geometry().buffer(1)

    # Candidate glaciers within search buffer (meters)
    search = lake_geom.buffer(ee.Number(buffer_km).multiply(1000))
    nearby = glaciers.filterBounds(search)

    # Null-like fields helper
    def _no_hits(f):
        return ee.Feature(f).set({
            "glacier_contact": False,
            "glacier_touch_count": 0,
            "nearest_glacier_dist_m": None,
            "lake_elev_m": None,
            "glacier_elev_m": None,
            "slope_glac_to_lake": None,
            "glacier_area_ha": None,
            "glacier_ids": ee.List([]),
            "buffer_km_used": buffer_km,
            "slope_method": "min_segment_poly_to_poly"
        })

    def _compute(f):
        # 1) Lake elevation for filtering (coarse, at centroid)
        lake_center       = lake_geom.centroid()
        lake_elev_filter  = dem.sample(lake_center, 30).first().get("elevation")

        # 2) Add distance/area + coarse glacier elevation (at glacier centroid) to each candidate
        def add_metrics(g):
            gg = ee.Feature(g).geometry()
            glac_elev_est = dem.sample(gg.centroid(), 30).first().get("elevation")
            return ee.Feature(g).set({
                "dist": gg.distance(lake_geom, 1),            # min poly↔poly distance (m), 1m tol
                "glacier_area_ha": gg.area(1).divide(1e4),    # hectares
                "glac_elev_est": glac_elev_est
            })
        with_metrics = nearby.map(add_metrics)

        # 3) Keep only glaciers strictly higher than lake and NON-TOUCHING
        higher          = with_metrics.filter(ee.Filter.gt("glac_elev_est", lake_elev_filter))
        higher_non_touch= higher.filter(ee.Filter.gt("dist", 0))

        # 4) If no valid candidate -> null-like
        return ee.Feature(ee.Algorithms.If(
            higher_non_touch.size().eq(0),
            _no_hits(f),
            # 5) Use nearest among higher & non-touching
            (lambda nearest:
                # Closest points on each polygon (the min-distance segment)
                (lambda lake_pt, glac_pt:
                    # Sample DEM at those closest points (30 m; SRTM native)
                    (lambda lake_elev, glac_elev:
                        # Distance between the two closest points (equals min poly distance)
                        (lambda min_dist:
                            # Slope = (glacier - lake) / run  (guard run > 0)
                            (lambda slope:
                                ee.Feature(f).set({
                                    "glacier_contact":        False,  # non-touching by construction
                                    "glacier_touch_count":    0,
                                    "nearest_glacier_dist_m": min_dist,
                                    "lake_elev_m":            lake_elev,
                                    "glacier_elev_m":         glac_elev,
                                    "slope_glac_to_lake":     slope,
                                    "glacier_area_ha":        nearest.get("glacier_area_ha"),
                                    "glacier_ids":            ee.List([nearest.get("glac_id")]),
                                    "buffer_km_used":         buffer_km,
                                    "slope_method":           "min_segment_poly_to_poly"
                                })
                            )(
                                ee.Algorithms.If(
                                    min_dist.gt(0),
                                    ee.Number(glac_elev).subtract(ee.Number(lake_elev)).divide(min_dist),
                                    ee.Number(0)
                                )
                            )
                        )(
                            ee.Geometry(lake_pt).distance(glac_pt, 1)  # 1m tol
                        )
                    )(
                        dem.sample(lake_pt, 30).first().get("elevation"),
                        dem.sample(glac_pt, 30).first().get("elevation")
                    )
                )(
                    lake_geom.closestPoint(nearest.geometry(), 1),
                    nearest.geometry().closestPoint(lake_geom, 1)
                )
            )(
                ee.Feature(ee.FeatureCollection(higher_non_touch).sort("dist").first())
            )
        ))

    # Top-level: if none in buffer at all -> null-like; else compute
    return ee.Feature(
        ee.Algorithms.If(nearby.size().eq(0), _no_hits(lake), _compute(lake))
    )
# --- 2) Public: map over a FeatureCollection whose geometry is a LAKE POLYGON (or point) ---
def annotate_glacier_metrics_from_polygons(lakes_fc, buffer_km=50):
    """
    Input:  FeatureCollection where geometry is the LAKE polygon.
            (If some are points, they are buffered 1 m internally so distance works.)
    Output: Same features with glacier metrics added as properties (server-side).
    """
    return ee.FeatureCollection(
        ee.FeatureCollection(lakes_fc).map(
            lambda f: _glacier_metrics_for_lake_polygon(ee.Feature(f), buffer_km)
        )
    )

# DataFrame -> FC of points (has 'id' uniqued in df_to_fc)
import numpy as np
import pandas as pd

df_safe = df_pos_expansion10y5row.copy()
# Replace NaN and +/-inf with None (EE → null)
df_safe = df_safe.replace([np.nan, np.inf, -np.inf], None)

fc_points = df_to_fc(df_safe)

# Map your lake-detection wrapper to get polygons with area property
fc_polys = fc_points.map(detect_area_glof)

# buffer_km can be tuned; 50 km is your default
fc_polys_with_glac = annotate_glacier_metrics_from_polygons(fc_polys, buffer_km=5)
df_glac_props = fc_properties_to_df(fc_polys_with_glac, max_items=len(df_pos_expansion10y5row))

# Keep only the columns you care about
df_glac_trial = df_glac_props[[
    "glacier_area_ha",             # glacier area in hectares
    "slope_glac_to_lake",          # slope from glacier to lake
    "glacier_contact",             # True/False if touching
    "glacier_touch_count",         # number of glaciers directly touching
    "nearest_glacier_dist_m",      # distance to nearest glacier (m)
    "glacier_elev_m",               # glacier elevation (m)
    "Latitude",
    "Longitude",
    "Year_final"

]]
df_glac_trial.head()


'''no exception. reason ig took too much time to compute'''

In [ ]:
#reason:::

import ee

def _glacier_metrics_for_lake_polygon(lake, buffer_km=50):
    """
    Annotate ONE lake polygon with glacier metrics.

    Logic:
      - If touching glaciers exist: mark contact=True, count them, sum area, slope=0.
      - Else: among non-touching glaciers, keep only those with glac_cp_elev > lake_cp_elev,
              pick the nearest, compute slope along the min-distance segment.
      - If no valid glacier -> return null-like fields.
    """
    glaciers = ee.FeatureCollection("GLIMS/current")
    dem      = ee.Image("USGS/SRTMGL1_003")  # 30 m DEM

    lake_geom = ee.Feature(lake).geometry().buffer(1)

    search = lake_geom.buffer(ee.Number(buffer_km).multiply(1000))
    nearby = glaciers.filterBounds(search)

    def _no_hits(f):
        return ee.Feature(f).set({
            "glacier_contact": False,
            "glacier_touch_count": 0,
            "nearest_glacier_dist_m": None,
            "lake_elev_m": None,
            "glacier_elev_m": None,
            "slope_glac_to_lake": None,
            "glacier_area_ha": None,
            "glacier_ids": ee.List([]),
            "buffer_km_used": buffer_km,
            "slope_method": "closest_points_min_segment"
        })

    def _compute(f):
        # --- Add metrics per glacier ---
        def add_metrics(g):
            g  = ee.Feature(g)
            gg = g.geometry()

            lake_pt = lake_geom.closestPoint(gg, 1)
            glac_pt = gg.closestPoint(lake_geom, 1)

            lake_cp_elev = ee.Number(dem.sample(lake_pt, 30).first().get("elevation"))
            glac_cp_elev = ee.Number(dem.sample(glac_pt, 30).first().get("elevation"))

            min_dist_cp  = ee.Number(ee.Geometry(lake_pt).distance(glac_pt, 1))
            area_ha      = ee.Number(gg.area(1)).divide(1e4)

            return g.set({
                "lake_cp_elev":  lake_cp_elev,
                "glac_cp_elev":  glac_cp_elev,
                "min_dist_cp":   min_dist_cp,
                "glacier_area_ha": area_ha,
                "glac_id":       g.get("glac_id"),
                "GLIMS_ID":      g.get("GLIMS_ID")
            })

        with_metrics = nearby.map(add_metrics)

        # --- Touching glaciers branch (dist==0) ---
        touching = with_metrics.filter(ee.Filter.eq("min_dist_cp", 0))

        def _touching_case():
            touch_ids  = touching.aggregate_array("glac_id")
            touch_area = ee.Number(touching.aggregate_sum("glacier_area_ha"))
            touch_count= touching.size()

            return ee.Feature(f).set({
                "glacier_contact":        True,
                "glacier_touch_count":    touch_count,
                "nearest_glacier_dist_m": 0,
                "lake_elev_m":            None,
                "glacier_elev_m":         None,
                "slope_glac_to_lake":     0,
                "glacier_area_ha":        touch_area,
                "glacier_ids":            touch_ids,
                "buffer_km_used":         buffer_km,
                "slope_method":           "touching"
            })

        # --- Non-touching branch ---
        def _non_touching_case():
            non_touch = with_metrics.filter(ee.Filter.gt("min_dist_cp", 0))

            # Add validity flag (glacier CP elev > lake CP elev)
            def add_valid_flag(g):
                return ee.Feature(g).set(
                    "is_valid",
                    ee.Number(g.get("glac_cp_elev")).gt(ee.Number(g.get("lake_cp_elev")))
                )

            with_valid = non_touch.map(add_valid_flag)
            valid = with_valid.filter(ee.Filter.eq("is_valid", 1))

            def _no_valid():
                return _no_hits(f)

            def _emit_final():
                best = ee.Feature(ee.FeatureCollection(valid).sort("min_dist_cp").first())

                min_dist     = ee.Number(best.get("min_dist_cp"))
                lake_elev_cp = ee.Number(best.get("lake_cp_elev"))
                glac_elev_cp = ee.Number(best.get("glac_cp_elev"))

                slope = ee.Algorithms.If(
                    min_dist.gt(0),
                    glac_elev_cp.subtract(lake_elev_cp).divide(min_dist),
                    ee.Number(0)
                )

                chosen_id = ee.Algorithms.If(
                    best.get("glac_id"),
                    best.get("glac_id"),
                    best.get("GLIMS_ID")
                )

                return ee.Feature(f).set({
                    "glacier_contact":        False,
                    "glacier_touch_count":    0,
                    "nearest_glacier_dist_m": min_dist,
                    "lake_elev_m":            lake_elev_cp,
                    "glacier_elev_m":         glac_elev_cp,
                    "slope_glac_to_lake":     slope,
                    "glacier_area_ha":        best.get("glacier_area_ha"),
                    "glacier_ids":            ee.List([chosen_id]),
                    "buffer_km_used":         buffer_km,
                    "slope_method":           "closest_points_min_segment"
                })

            return ee.Feature(ee.Algorithms.If(valid.size().eq(0), _no_valid(), _emit_final()))

        return ee.Feature(
            ee.Algorithms.If(touching.size().gt(0), _touching_case(), _non_touching_case())
        )

    return ee.Feature(
        ee.Algorithms.If(nearby.size().eq(0), _no_hits(lake), _compute(lake))
    )

# --- 2) Public: map over a FeatureCollection whose geometry is a LAKE POLYGON (or point) ---
def annotate_glacier_metrics_from_polygons(lakes_fc, buffer_km=10):
    """
    Input:  FeatureCollection where geometry is the LAKE polygon.
            (If some are points, they are buffered 1 m internally so distance works.)
    Output: Same features with glacier metrics added as properties (server-side).
    """
    return ee.FeatureCollection(
        ee.FeatureCollection(lakes_fc).map(
            lambda f: _glacier_metrics_for_lake_polygon(ee.Feature(f), buffer_km)
        )
    )

# DataFrame -> FC of points (has 'id' uniqued in df_to_fc)
import numpy as np
import pandas as pd

df_safe = df_pos_expansion10y5row.copy()
# Replace NaN and +/-inf with None (EE → null)
df_safe = df_safe.replace([np.nan, np.inf, -np.inf], None)

fc_points = df_to_fc(df_safe)

# Map your lake-detection wrapper to get polygons with area property
fc_polys = fc_points.map(detect_area_glof)

# buffer_km can be tuned; 50 km is your default
fc_polys_with_glac = annotate_glacier_metrics_from_polygons(fc_polys, buffer_km=10)
df_glac_props = fc_properties_to_df(fc_polys_with_glac, max_items=len(df_pos_expansion10y5row))

# Keep only the columns you care about
df_glac_trial = df_glac_props[[
    "glacier_area_ha",             # glacier area in hectares
    "slope_glac_to_lake",          # slope from glacier to lake
    "glacier_contact",             # True/False if touching
    "glacier_touch_count",         # number of glaciers directly touching
    "nearest_glacier_dist_m",      # distance to nearest glacier (m)
    "glacier_elev_m",               # glacier elevation (m)
    "Latitude",
    "Longitude",
    "Year_final"

]]
df_glac_trial.head()


##reason of cancellation:
'''EEException: User memory limit exceeded.'''